In [17]:
import sys
import torch
import torch.nn as nn
from torch.autograd import Variable
import librosa
import torch.nn.functional as F
from Modules.SpectralProcessing import PreNet
from  Modules.Conv1dGLU import Conv1dGLU
from Modules.Encoder import Attention

In [18]:
batch_size = 25
N_samples = 20

In [19]:
class Encoder(nn.Module):
    global batch_size
    global N_samples
    def __init__(self):
        super(Encoder, self).__init__()
        self.prenet = PreNet()
        self.conv = Conv1dGLU()
        self.attention = Attention(128)
        self.prohead = nn.Linear(128,1)
        self.residual_conv = nn.Linear(128,512)
        self.bn = nn.BatchNorm1d(N_samples)
        
    def forward(self, x):
        x = self.prenet(x)
        x = x.view(batch_size*N_samples, x.size(2), x.size(3)).transpose(1,2)
        x = self.conv(x)
        x = x.transpose(1,2)
        x.contiguous()
        x = x.view(batch_size,N_samples,x.size(1),x.size(2))
        #x = Temp_Masking(x)
        x = x.mean(dim=2)
        conv_out = x
        conv_out = self.residual_conv(conv_out)
        x = self.attention(x)
        x = self.prohead(x)
        x = x.squeeze()
        x = F.softsign(x)
        x = self.bn(x)
        x = x.unsqueeze(dim=2)
        x = torch.bmm(x.transpose(1,2), conv_out)
        x = x.squeeze()
        return x

In [20]:
enc = Encoder()

In [25]:
z = torch.rand(25,20,100,80)

In [26]:
out = enc(Variable(z))
out

Variable containing:
 1.4653e-01 -3.3818e-01  5.9525e-02  ...  -2.0935e-01 -4.5897e-02 -3.1991e-01
 1.8477e-01 -3.6472e-01  1.0381e-02  ...  -2.1133e-01 -3.5640e-02 -3.3922e-01
 2.9766e-01 -6.0187e-01  9.7640e-02  ...  -4.1723e-01 -6.6396e-02 -6.1291e-01
                ...                   ⋱                   ...                
 2.6267e-01 -5.3324e-01  8.8244e-02  ...  -3.9163e-01 -5.5629e-02 -5.4774e-01
 2.0818e-01 -4.0343e-01  7.4120e-02  ...  -2.7109e-01 -6.7430e-02 -4.0250e-01
-2.3952e-01  5.2360e-01 -1.1327e-01  ...   3.3019e-01  7.7803e-02  4.9992e-01
[torch.FloatTensor of size 25x512]

In [27]:
def Temp_Masking(x):
    final = torch.zeros_like(x)
    for i in range(batch_size):
        for j in range(N_samples):
            temp = np.array(x[i,j,:,:])
            temp = librosa.decompose.hpss(temp)[0]
            final[i,j]+=torch.from_numpy(temp)
    return Variable(final)

In [28]:
Temp_Masking(z)

Variable containing:
( 0 , 0 ,.,.) = 
  1.4197e-01  1.9604e-01  2.9105e-01  ...   4.2646e-01  4.7981e-01  2.6327e-01
  1.1905e-01  5.2704e-03  1.5112e-01  ...   1.6074e-02  3.1682e-01  3.5424e-01
  4.0943e-01  2.0413e-01  2.5102e-01  ...   2.8681e-01  1.3191e-01  1.0281e-01
                 ...                   ⋱                   ...                
  1.3973e-01  4.2458e-02  8.4805e-02  ...   2.0258e-01  3.6510e-01  9.3438e-02
  3.6031e-01  1.8441e-01  3.2177e-01  ...   8.1308e-02  3.8688e-02  2.7203e-01
  2.3573e-01  3.9225e-01  9.1034e-02  ...   6.0427e-01  5.8773e-01  8.6162e-01

( 0 , 1 ,.,.) = 
  2.5327e-01  3.2528e-01  7.1061e-02  ...   3.5228e-01  1.2596e-01  1.2461e-02
  4.7734e-01  2.6880e-01  3.4708e-01  ...   5.2709e-01  7.2015e-02  5.9878e-01
  4.0917e-01  2.7616e-01  4.4394e-01  ...   6.9525e-01  6.7678e-01  6.4374e-01
                 ...                   ⋱                   ...                
  6.2174e-01  5.2218e-02  3.0286e-01  ...   3.3563e-01  7.3664e-02  2.6702e

In [29]:
#Temp Masking not happening because some values of input are negative